# Лабораторная работа 3. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы ФИО ГРУППА
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена

флойд o(n**3) - количество циклов по числу вершин, возможность использования отрциательных значений для ребер (но не отриц циклов) , сначала ищет кратчайший путь а потом другие через промежуточные вершины
декстра (O((V + E) log V)) - поддерживает только полодит знач для графа берет и сравнивает знач до следующих вершин где меньше туда и идет
Беллмана-Форда  O(V·E) - большая сложность проходит циклами v - 1(кол -во вершин) по сути декстра но работает с отриц знач и перепроверяет все возмож варианты(с перезаписью если лучше)
Джонсона -  ипользует комбинацию Беллмана-Форда + Дейкстры с небольшим трюком: перевзвешивание рёбер (reweighting), чтобы все рёбра стали положительными — и тогда можно безопасно использовать Дейкстру.
левита -  это алгоритмы декстры который умеет откатываться назад если нашел более оптимальный путь

йена - Сначала находит обычный кратчайший путь (например, с помощью Дейкстры) — это путь №1.

Потом по очереди "сворачивает" с найденного пути, чтобы найти другие варианты:

Берёт начальную часть пути (root),

Убирает одно из рёбер, чтобы пройти по-другому (spur),

Склеивает новую дорогу и сохраняет её.

Из всех новых вариантов выбирает самый короткий — это путь №2.

Повторяет процесс, пока не найдёт все k путей.


### Выполнение:

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time

class GraphApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Поиск кратчайшего пути — Алгоритм Дейкстры")

        self.graph = nx.DiGraph()

        self.setup_ui()

    def setup_ui(self):
        control_frame = tk.Frame(self.root)
        control_frame.pack(side=tk.TOP, fill=tk.X)

        tk.Button(control_frame, text="Загрузить граф", command=self.load_graph).pack(side=tk.LEFT)
        tk.Label(control_frame, text="Начальная вершина:").pack(side=tk.LEFT)
        self.start_entry = tk.Entry(control_frame, width=5)
        self.start_entry.pack(side=tk.LEFT)

        tk.Label(control_frame, text="Конечная вершина:").pack(side=tk.LEFT)
        self.end_entry = tk.Entry(control_frame, width=5)
        self.end_entry.pack(side=tk.LEFT)

        tk.Button(control_frame, text="Найти путь", command=self.find_shortest_path).pack(side=tk.LEFT)
        
        # веришины
        self.figure = plt.Figure(figsize=(6, 6))
        self.ax = self.figure.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.figure, self.root)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    def load_graph(self):
        file_path = filedialog.askopenfilename()
        if not file_path:
            return

        self.graph.clear()
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                weights = list(map(int, line.strip().split()))
                for j, weight in enumerate(weights):
                    if weight != 0:
                        self.graph.add_edge(str(i), str(j), weight=weight)

        self.draw_graph()

    def draw_graph(self, path_edges=None):
        self.ax.clear()
        pos = nx.spring_layout(self.graph)
        edge_labels = nx.get_edge_attributes(self.graph, 'weight')
        
        nx.draw(self.graph, pos, ax=self.ax, with_labels=True, node_color='lightblue', edge_color='gray')
        nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=edge_labels, ax=self.ax)

        if path_edges:
            nx.draw_networkx_edges(self.graph, pos, edgelist=path_edges, edge_color='red', width=2, ax=self.ax)

        self.canvas.draw()

    def find_shortest_path(self):
        start = self.start_entry.get()
        end = self.end_entry.get()
        if start not in self.graph.nodes or end not in self.graph.nodes:
            messagebox.showerror("Ошибка", "Некорректные вершины")
            return

        try:
            start_time = time.time()
            path = nx.dijkstra_path(self.graph, source=start, target=end, weight='weight')
            total_weight = nx.dijkstra_path_length(self.graph, source=start, target=end, weight='weight')
            elapsed_time = time.time() - start_time

            path_edges = list(zip(path, path[1:])) 
            self.draw_graph(path_edges)
            messagebox.showinfo("Результат", f"Кратчайший путь: {' → '.join(path)}\nДлина: {total_weight}\nВремя: {elapsed_time:.6f} сек")

        except nx.NetworkXNoPath:
            messagebox.showerror("Ошибка", "Путь не найден")


if __name__ == "__main__":
    root = tk.Tk()
    app = GraphApp(root)
    root.mainloop()


### Вывод